## 1. Connect to SQL and get list of countries.

In [12]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pyodbc 

conn = pyodbc.connect(
    "DRIVER={SQL Server};" 
    "SERVER=DESKTOP-ID946DB\\SQLEXPRESS01;"
    "DATABASE=FSI-PROJECT;" 
    "Trusted_Connection=yes;" 
)

cursor = conn.cursor() # Cursor for queries

## 2. Retrieve Country List from SQL

In [52]:
cursor.execute("SELECT DISTINCT country FROM dbo.FSI_NORMALIZED_PY") # Select unique country names

country_list = [row[0] for row in cursor.fetchall()] # Extract country name from each row

country_list[:5]

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda']

## 3. Install  **`country_converter`** library

In [25]:
!pip install country_converter 

import country_converter as coco

# Create a CountryConverter object 
cc = coco.CountryConverter() 

## 4. create a Pandas DataFrame. 
#### Now that we have a: 
##### ► List of countries and 
##### ► The converter ready, 
#### Let's create 
##### → A dataframe to add the continents column


In [72]:
import pandas as pd

df_geo_data = pd.DataFrame({'country': country_list}) # So, this creates a dataframe with a country column

# Generate continent list and Convert country names to continents
continent_list = cc.convert(names=df_geo_data['country'].tolist(), to='continent')

# Add 'continent' column
df_geo_data['continent'] = continent_list # Assign continent list to 'continent' column

print(df_geo_data.head())

More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match for Israel and West Bank
More than one regular expression match f

               country continent
0          Afghanistan      Asia
1              Albania    Europe
2              Algeria    Africa
3               Angola    Africa
4  Antigua and Barbuda   America


## 5.  Update the SQL Table with Continent Data

In [61]:
for index, row in df_geo_data.iterrows():
    query = """
        UPDATE dbo.FSI_NORMALIZED_PY
        SET continent = ?
        WHERE country = ?
    """
    continent_value = str(row['continent']) 
    country_value = str(row['country'])      
    cursor.execute(query, (continent_value, country_value))  # Execute update with parameters

conn.commit()  # Save all updates

## 6. Verify Update Results in SQL
#### check for missing continent values in the DataFrame.


In [58]:
query = "SELECT COUNT(*) FROM dbo.FSI_NORMALIZED_PY WHERE continent IS NOT NULL"
cursor.execute(query)
print(f" ► Total rows updated: {cursor.fetchone()[0]}")


 ► Total rows updated: 3170


In [70]:
query = """
SELECT COUNT(DISTINCT country)
FROM dbo.FSI_NORMALIZED_PY
WHERE continent IS NOT NULL
"""
cursor.execute(query)
updated_unique_countries = cursor.fetchone()[0]

unique_countries_df = df_geo_data['country'].nunique()

print(f"Unique countries updated in SQL: {updated_unique_countries}")
print(f"Unique countries in DataFrame: {unique_countries_df}")



Unique countries updated in SQL: 187
Unique countries in DataFrame: 187
